#### Parametri da modificare:

In [14]:
# Specificare la cartella da analizzare con i programmi sas
cartella_da_esplorare = r"C:\Miguel\progetti\05_Modifica Testi\01_data\TEST\PROVA ELIMINA"
estensione_file = ".sas"

# Specificare il percorso e file excel di output
excel_file_path = r"C:\Miguel\progetti\05_Modifica Testi\02_output\ReportPrincipale.xlsx"

# 1) Funzioni per cercare parole:
***INPUT:***

- cartella:type(str): cartella principale da analizzare
- estensione:type(str): estensione del file da elaborare 
- parole_da_verificare:type(list): lista di parole da cercare
- parole_da_escludere:type(list): lista di parole da escludere


***FASI:***

1) La funzione **crea_dataframe_presenza_parole** itera attraverso le cartelle, incluse le sottocartelle, alla ricerca di programmi con estensione .sas.
2) Per ciascun file .sas individuato, la funzione esamina tutte le righe del programma.
3) La funzione **elimina_parole_tra_commenti** elimina il testo compreso tra i commenti.
4) La funzione **verifica_presenza_parole** utilizza una condizione per verificare se tutte le parole nella lista **parole_da_verificare** sono presenti nella riga, contemporaneamente verificando l'assenza di qualsiasi parola dalla lista parole_da_escludere.
5) La funzione **crea_dataframe_presenza_parole** restituisce due output
   
    - Un DataFrame contenente il nome e la cartella del programma con tutti gli indici identificati.
    - Un DataFrame contenente il nome e la cartella del programma con il testo completo delle righe identificate



In [15]:
import os
import pandas as pd

def elimina_parole_tra_commenti(line, in_comment_block):
    if in_comment_block:
        end_comment = line.find('*/')
        if end_comment != -1:
            line = line[end_comment + 2:]
            in_comment_block = False
        else:
            return '', True  # continua a leggere la prossima riga

    start_comment = line.find('/*')
    while start_comment != -1:
        end_comment = line.find('*/', start_comment + 2)
        if end_comment != -1:
            line = line[:start_comment] + line[end_comment + 2:]
            start_comment = line.find('/*')
        else:
            in_comment_block = True
            line = line[:start_comment]
            break

    return line, in_comment_block

def verifica_presenza_parole(file_path, parole_da_verificare, parole_da_escludere, linee_con_parole):
    indici_presenti = []
    in_comment_block = False

    with open(file_path, 'r') as file:
        for numero, line in enumerate(file, start=1):
            line_originale = line.strip()

            # Elimina le parole tra /* e */
            line_senza_commenti, in_comment_block = elimina_parole_tra_commenti(line_originale, in_comment_block)

            # CONDIZIONE
            if all(parola.lower() in line_senza_commenti.lower() for parola in parole_da_verificare) and \
               all(parola.lower() not in line_senza_commenti.lower() for parola in parole_da_escludere):
                indici_presenti.append(numero)
                # linee_con_parole.append(line_originale.lower())
                
                file_nome = os.path.basename(file.name)
                root = os.path.dirname(file.name)
                linee_con_parole['Nome File'].append(file_nome)
                linee_con_parole['Percorso File'].append(root)
                linee_con_parole['Riga'].append(line_originale.lower())
                linee_con_parole['Indice'].append(numero)

    return indici_presenti

def crea_dataframe_presenza_parole(cartella, estensione, parole_da_verificare, parole_da_escludere):
    stringa_verificare = ','.join(parole_da_verificare)
    stringa_escludere = ','.join(parole_da_escludere)
    if len(parole_da_escludere) == 0:
        nome_colonna=stringa_verificare
    else:
        nome_colonna=stringa_verificare + ' EXCLUDE ' + stringa_escludere
    
    dati_temporanei = {'Nome File': [], 'Percorso File' : [], nome_colonna: []}
    linee_con_parole = {'Nome File': [], 'Percorso File' : [], 'Riga' :  [], 'Indice': []}

    for root, dirs, files in os.walk(cartella):
        for file_name in files:
            if file_name.endswith(estensione):
                file_path = os.path.join(root, file_name)
                indici_presenti = verifica_presenza_parole(file_path, parole_da_verificare, parole_da_escludere, linee_con_parole)
                dati_temporanei['Nome File'].append(file_name)
                dati_temporanei['Percorso File'].append(root)
                dati_temporanei[nome_colonna].append(indici_presenti)

    dataframe = pd.DataFrame(dati_temporanei)
    dataframe.set_index('Nome File', inplace=True)

    df_linee = pd.DataFrame(linee_con_parole)

    df_linee['Frequenza Programmi'] = df_linee.groupby('Riga')['Riga'].transform('count')
    df_linee.sort_values(by='Frequenza Programmi',ascending=False, inplace=True)
    df_linee.rename(columns = {'Riga':nome_colonna}, inplace=True)
    
    return dataframe, df_linee

Chiama la funzione per le seguenti parole:

connect to | proc import | proc export | %email | %let | call symput | %include | libname


In [16]:
# Chiama la funzione per creare il DataFrame
connect_to_pgm, connect_to_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['connect to'], [])

proc_import_pgm, proc_import_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['proc import'],[])

proc_export_pgm, proc_export_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['proc export'],[])

mail_pgm, mail_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['%email'],[])

let_pgm, let_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['%let'],[])

call_symput_pgm, call_symput_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['call symput'],[])

include_pgm, include_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['%include'],[])

libname_pgm, libname_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, ['libname'],["%include"])

Frequenza %let

In [17]:
# Frequenza dei nomi delle macro-variabili
# name_let = let_linee['%let'].apply(lambda x: x.split('=')[0])
# name_let = name_let.apply(lambda x: x.replace('%let ', ''))
# name_let = name_let.str.strip().str.replace('\\s+', ' ')
# name_let.value_counts()

# 2) Funzioni per cercare regex

***INPUT:***
- cartella:type(str): cartella principale da analizzare
- estensione:type(str): estensione del file da elaborare 
- regex_pattern:type(regex): esempio r'\bcreate table\b\s+([^\s]+\.[^\s]+\s+.*)'
- nome_colonna:type(str): nome delle colonna delle regex
- parole_da_escludere:type(list): lista di parole da escludere


***FASI:***

1) La funzione **crea_dataframe_presenza_parole** itera attraverso le cartelle, inclusi i sottocartelle, alla ricerca di programmi con estensione .sas.
2) Per ciascun file .sas individuato, la funzione esamina tutte le righe del programma.
3) La funzione **elimina_parole_tra_commenti** elimina il testo compreso tra i commenti.
4) La funzione **verifica_presenza_parole** utilizza una condizione per verificare se la regex_pattern è presente nella riga, contemporaneamente verificando l'assenza di qualsiasi parola dalla lista parole_da_escludere.
5) La funzione **crea_dataframe_presenza_parole** restituisce due output
   
    - Un DataFrame contenente il nome e la cartella del programma con tutti gli indici identificati.
    - Un DataFrame contenente il nome e la cartella del programma con il testo completo delle righe identificate


In [18]:
import os
import pandas as pd
import re

def elimina_parole_tra_commenti(line, in_comment_block):
    if in_comment_block:
        end_comment = line.find('*/')
        if end_comment != -1:
            line = line[end_comment + 2:]
            in_comment_block = False
        else:
            return '', True  # Continua a leggere la prossima riga

    start_comment = line.find('/*')
    while start_comment != -1:
        end_comment = line.find('*/', start_comment + 2)
        if end_comment != -1:
            line = line[:start_comment] + line[end_comment + 2:]
            start_comment = line.find('/*')
        else:
            in_comment_block = True
            line = line[:start_comment]
            break

    return line, in_comment_block

def verifica_presenza_parole(file_path, regex_pattern, parole_da_escludere, linee_con_parole):
    indici_presenti = []
    in_comment_block = False

    with open(file_path, 'r') as file:
        for numero, line in enumerate(file, start=1):
            line_originale = line.strip()

            # Elimina le parole tra /* e */
            line_senza_commenti, in_comment_block = elimina_parole_tra_commenti(line_originale, in_comment_block)
            # regex_pattern = r'\bdata\b\s+\S+\.\S+\s*;'
            
            # Verifica la presenza di tutte le parole nella lista e l'assenza delle parole da escludere
            if re.search(regex_pattern, line_senza_commenti.lower(), re.IGNORECASE) and \
               all(parola.lower() not in line_senza_commenti.lower() for parola in parole_da_escludere):
                indici_presenti.append(numero)
                # linee_con_parole.append(line_originale.lower())
                
                file_nome = os.path.basename(file.name)
                root = os.path.dirname(file.name)
                
                linee_con_parole['Nome File'].append(file_nome)
                linee_con_parole['Percorso File'].append(root)
                linee_con_parole['Riga'].append(line_originale.lower())
                linee_con_parole['Indice'].append(numero)
              
    return indici_presenti

def crea_dataframe_presenza_parole(cartella, estensione, nome_colonna, regex_pattern, parole_da_escludere):
 
    nome_colonna=nome_colonna
    dati_temporanei = {'Nome File': [], 'Percorso File' : [], nome_colonna: []}
    linee_con_parole = {'Nome File': [], 'Percorso File' : [], 'Riga' :  [], 'Indice': []}

    for root, dirs, files in os.walk(cartella):
        for file_name in files:
            if file_name.endswith(estensione):
                file_path = os.path.join(root, file_name)
                indici_presenti = verifica_presenza_parole(file_path, regex_pattern, parole_da_escludere, linee_con_parole)
                dati_temporanei['Nome File'].append(file_name)
                dati_temporanei['Percorso File'].append(root)
                dati_temporanei[nome_colonna].append(indici_presenti)

    dataframe = pd.DataFrame(dati_temporanei)
    dataframe.set_index('Nome File', inplace=True)

    df_linee = pd.DataFrame(linee_con_parole)
    df_linee['Frequenza Programmi'] = df_linee.groupby('Riga')['Riga'].transform('count')
    df_linee.sort_values(by='Frequenza Programmi',ascending=False, inplace=True)
    df_linee.rename(columns = {'Riga':nome_colonna}, inplace=True)

    return dataframe, df_linee

### Dove scrive...

#### DATA

In [19]:
DATA_pgm, DATA_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, "Statement DATA=", r'\bdata\b\s+(\S+\.\S+(?:\s+\S+)*)\s*;',["work."])

In [20]:
DATA_linee

,Nome File,Percorso File,Statement DATA=,Indice,Frequenza Programmi
192,Parte 2 - DataCollection_MST_ID_test.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data new_er.volvo_mildhyb;,2524,7
91,Parte 2 - DataCollection_MST_ID.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data new_er.volvo_mildhyb;,2209,7
92,Parte 2 - DataCollection_MST_ID.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data new_er.volvo_mildhyb;,2230,7
142,Parte 2 - DataCollection_MST_ID_202305.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data new_er.volvo_mildhyb;,2210,7
143,Parte 2 - DataCollection_MST_ID_202305.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data new_er.volvo_mildhyb;,2231,7
...,...,...,...,...,...
55,check_LCV_segmento_202002.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data /*dati.*/gt_cil16_&ambiente._1;,961,1
54,check_LCV_segmento_202002.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data /*dati.*/flag4gov_&ambiente._1;,904,1
53,check_LCV_segmento_202002.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data /*dati.*/gt_compsw_&ambiente._1;,767,1
52,check_LCV_segmento_202002.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,data /*dati.*/gt_ep_&ambiente._1;,681,1


#### CREATE TABLE

In [21]:
CREATE_TABLE_pgm, CREATE_TABLE_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, "Statement CREATE TABLE=", r'\bcreate table\b\s+([^\s]+\.[^\s]+\s+.*)', ["work."])

In [22]:
CREATE_TABLE_linee

,Nome File,Percorso File,Statement CREATE TABLE=,Indice,Frequenza Programmi
53,SMRv2 Parte 2 + DataCollection_MST_ID.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,create table &outable.parms as select parms fr...,67,12
84,Pt01 - Quotation_new_202202.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,create table &outable.parms as select parms fr...,450,12
72,SMRv2 Parte 2 + DataCollection_MST_ID.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,create table &outable.parms as select parms fr...,67,12
61,SMRv2_DataCollection_MST_ID_UAT.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,create table &outable.parms as select parms fr...,67,12
60,SMRv2 Parte 4 - DataCollection_REGISTRATION.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,create table &outable.parms as select parms fr...,454,12
...,...,...,...,...,...
23,0221_DRI_Ins_DWH_Suppliers_v13.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,create table dwh_&ambiente..dri_dwh_suppliers_...,1317,1
22,0221_DRI_Ins_DWH_Suppliers_v13.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,create table dwh_&ambiente..dri_dwh_suppliers_...,1310,1
21,0221_DRI_Ins_DWH_Suppliers_v13.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,create table dwh_&ambiente..dri_dwh_suppliers as,1206,1
20,0221_DRI_Ins_DWH_Suppliers_v13.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,create table dwh_&ambiente..dri_dwh_sup_attrib...,326,1


In [23]:
FROM_pgm, FROM_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, "Statement=", r'\bfrom\b\s+([^\s]+\.[^\s]+\s+.*)', ["work."])

In [24]:
INNER_JOIN_pgm, INNER_JOIN_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, "Statement=", r'\binner join\b\s+([^\s]+\.[^\s]+\s+.*)', ["work."])

In [25]:
LEFT_JOIN_pgm, LEFT_JOIN_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, "Statement=", r'\bleft join\b\s+([^\s]+\.[^\s]+\s+.*)', ["work."])

In [26]:
SET_pgm, SET_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, "Statement=",r'\bset\b\s+(\S+\.\S+(?:\s+\S+)*)\s*',["work."])

### Dove legge

#### SET

In [27]:
SET_pgm, SET_linee = crea_dataframe_presenza_parole(cartella_da_esplorare, estensione_file, "Statement SET=",r'\bset\b\s+(\S+\.\S+(?:\s+\S+)*)\s*',["work."])

In [28]:
SET_linee

,Nome File,Percorso File,Statement SET=,Indice,Frequenza Programmi
273,Parte 2 - DataCollection_MST_ID.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,set new_er.expertgovernancepercfuel;,2894,12
89,Parte 2 - DataCollection_MST_ID.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,set gpl_metano_&ambiente.;,1844,12
271,Parte 2 - DataCollection_MST_ID.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,set new_er.expertgovernancepercfuel;,2864,12
100,Parte 2 - DataCollection_MST_ID.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,set new_er.expertgovernancepercfuel;,2844,12
272,Parte 2 - DataCollection_MST_ID.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,set new_er.expertgovernancepercfuel;,2887,12
...,...,...,...,...,...
230,SMRv2 Parte 3a - CHECK EXPORT.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,set datismr.toolkit_bev;,314,1
318,Pt01 - Quotation_new_opt.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,set dwh_&ambiente..dri_dwh_vehicle_catalogue_3...,554,1
321,PRD_0378_DRI_Ins_DWH_Completo_Agile_Zone_v3.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,set dwh_&ambiente..dri_dwh_lease_prechange;,234,1
322,PRD_0378_DRI_Ins_DWH_Completo_Agile_Zone_v3.0.sas,C:\Miguel\progetti\05_Modifica Testi\01_data\T...,set dwh_&ambiente..dri_dwh_customer_policy_det...,288,1


### Merge Unico e Statistiche di base

Di seguito vengono concatenati tutti i dataframe di output, e creato un report con le statistiche di base

In [29]:
report_all = pd.concat([connect_to_pgm,  proc_import_pgm, proc_export_pgm, mail_pgm, let_pgm, call_symput_pgm, include_pgm, libname_pgm,
            DATA_pgm, SET_pgm
          ], axis=1)

report_all  = report_all.loc[:,~report_all.columns.duplicated()]


In [30]:
report_length = report_all.map(lambda x: len(x) if isinstance(x, list) else x)
totali_per_colonna = report_all.drop(columns='Percorso File').map(lambda x: len(x) if isinstance(x, list) else 0).sum()
conteggi_per_colonna = (report_all.drop(columns='Percorso File').map(lambda x: len(x) if isinstance(x, list) else 0) != 0).sum()

report_means = pd.DataFrame({
    'Keyword': totali_per_colonna.index,
    'Numero Programmi' : report_all.shape[0],
    'Frequenza Totale': totali_per_colonna.values,
    'Frequenza Programmi' : conteggi_per_colonna.values,
    'Percentuale Programmi' : conteggi_per_colonna.values/report_all.shape[0]
})

In [31]:
report_means

,Keyword,Numero Programmi,Frequenza Totale,Frequenza Programmi,Percentuale Programmi
0,connect to,129,118,45,0.348837
1,proc import,129,90,33,0.255814
2,proc export,129,236,54,0.418605
3,%email,129,37,35,0.271318
4,%let,129,302,93,0.720930
5,call symput,129,266,46,0.356589
6,%include,129,51,9,0.069767
7,libname EXCLUDE %include,129,15,12,0.093023
8,Statement DATA=,129,367,44,0.341085
9,Statement SET=,129,353,51,0.395349


### Export Output Finale

In [32]:
import pandas as pd

with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
    # Salva i DataFrame nei fogli del xlsx.
    report_means.to_excel(writer, sheet_name='COVER', index=False)
    report_all.reset_index().to_excel(writer, sheet_name='Lista PGM_v1.0', index=False)
    report_length.reset_index().to_excel(writer, sheet_name='Lista PGM_v2.0', index=False)
    DATA_linee.to_excel(writer, sheet_name='OUTPUT', index=False)
    CREATE_TABLE_linee.to_excel(writer, sheet_name='CREATE TABLE', index=False)
    SET_linee.to_excel(writer, sheet_name='INPUT', index=False)
    
    let_linee.to_excel(writer, sheet_name='%LET', index=False)
    call_symput_linee.to_excel(writer, sheet_name='CALL SYMPUT', index=False)
    include_linee.to_excel(writer, sheet_name='INCLUDE', index=False)
    libname_linee.to_excel(writer, sheet_name='LIBNAME', index=False)